In [1]:
import pysam
import pandas as pd
import glob

In [10]:
# PARAMETERS
# hmba_vcf_dir = "/anvil/projects/x-mcb130189/Wubin/BG/vcf"
# uci_vcf_dir = "/anvil/scratch/x-rwang22/bican/vcf_tmp"
# snps = "/anvil/projects/x-mcb130189/rwang22/bican/scripts/ldsc/ct_specific_dmr_signif_snps_overlap.tsv"


vcf_dir = "/anvil/projects/x-mcb130189/Wubin/BG/vcf"
snps = "/anvil/projects/x-mcb130189/rwang22/bican/scripts/ldsc/genetic_variant/Subclass.abc_signif_snps_overlap.tsv.tsv"
# snps = "/anvil/projects/x-mcb130189/rwang22/bican/scripts/ldsc/genetic_variant/Group.abc_signif_snps_overlap.tsv.tsv"

In [11]:
vcfs = ["H1930001.ann.vcf.gz", 
        "H1930002.ann.vcf.gz",
        "H1930004.ann.vcf.gz",
        "NBB9087.ann.vcf.gz", 
        "UCI2424.ann.vcf.gz",
        "UCI4723.ann.vcf.gz", 
        "UCI5224.ann.vcf.gz"]

In [12]:
# read in GWAS snps
snps_df = pd.read_csv(snps, sep = "\t")[["CHR_ID", "START", "END", "SNPS"]].drop_duplicates()
snps_df.head()

,CHR_ID,START,END,SNPS
0,chr1,2438058,2438059,rs4592207
1,chr1,2443318,2443319,rs4648844
2,chr1,2444404,2444405,rs6687012
3,chr1,8372075,8372076,rs894875
4,chr1,8409223,8409224,rs301790


In [13]:
genotypes = pd.DataFrame(index=[s for s in snps_df["SNPS"]], columns=vcfs)

In [14]:
for vcf_name in vcfs:
    if vcf_name[0] == "H":
        print(vcf_name[0])

H
H
H


In [15]:
for vcf_name in vcfs:
    # if vcf_name[0] == "H":
    #     vcf_path = f'{hmba_vcf_dir}/{vcf_name}'
    # else:
    #     vcf_path = f'{uci_vcf_dir}/{vcf_name}'
    vcf_path = f'{vcf_dir}/{vcf_name}'

    vcf = pysam.VariantFile(vcf_path)
    print(vcf_name)
    for idx, row in snps_df.iterrows():
        
        chrom = row["CHR_ID"]
        start = int(row["START"])
        end = int(row["END"])
        snp_id = row["SNPS"]

        records = list(vcf.fetch(chrom, end - 1, end))  # fetch around position

        if records:
            rec = records[0]
            gt = rec.samples[0]['GT']  # first (and only) sample in a single-sample VCF
            if gt == (0,0):
                call = "0"
            elif gt == (0,1) or gt == (1,0):
                call = "1"
            elif gt == (1,1):
                call = "2"
            else:
                call = "./."
            genotypes.loc[snp_id, vcf_name] = call
        else:
            genotypes.loc[snp_id, vcf_name] = "./."  # missing

H1930001.ann.vcf.gz
H1930002.ann.vcf.gz
H1930004.ann.vcf.gz


[W::hts_idx_load3] The index file is older than the data file: /anvil/projects/x-mcb130189/Wubin/BG/vcf/NBB9087.ann.vcf.gz.tbi


NBB9087.ann.vcf.gz


[W::hts_idx_load3] The index file is older than the data file: /anvil/projects/x-mcb130189/Wubin/BG/vcf/UCI2424.ann.vcf.gz.tbi


UCI2424.ann.vcf.gz
UCI4723.ann.vcf.gz
UCI5224.ann.vcf.gz


In [16]:
genotypes.head()

,H1930001.ann.vcf.gz,H1930002.ann.vcf.gz,H1930004.ann.vcf.gz,NBB9087.ann.vcf.gz,UCI2424.ann.vcf.gz,UCI4723.ann.vcf.gz,UCI5224.ann.vcf.gz
rs4592207,1,2,1,./.,./.,2,./.
rs4648844,1,2,1,2,./.,2,./.
rs6687012,1,2,1,2,./.,2,./.
rs894875,1,1,1,2,1,1,2
rs301790,1,1,1,./.,1,1,./.


In [17]:
# genotypes.to_csv("Group.abc_signif_snp_genotypes.tsv", sep = "\t", index = True)
genotypes.to_csv("Subclass.abc_signif_snp_genotypes.tsv", sep = "\t", index = True)